#Data

In [200]:
import pandas as pd

In [632]:
from preprocessing import preprocess

In [633]:
dfs = []
dfs.append(preprocess('../data/raw/CA/CA_1981-1985.txt'))
dfs.append(preprocess('../data/raw/CA/CA_1985-1989.txt'))
dfs.append(preprocess('../data/raw/CA/CA_1989-1993.txt'))
dfs.append(preprocess('../data/raw/CA/CA_1993-1997.txt'))
dfs.append(preprocess('../data/raw/CA/CA_1997-2001.txt'))
dfs.append(preprocess('../data/raw/CA/CA_2001-2005.txt'))
dfs.append(preprocess('../data/raw/CA/CA_2005-2009.txt'))
dfs.append(preprocess('../data/raw/CA/CA_2009-2015.txt'))

In [634]:
df = pd.concat(dfs)

In [635]:
df = df.dropna(axis=0)

In [636]:
df[['STN','YEARMODA','TEMP','DEWP','VISIB','WDSP','GUST','MAX','MIN','PRCP']].head(10)

,STN,YEARMODA,TEMP,DEWP,VISIB,WDSP,GUST,MAX,MIN,PRCP
14239,722900,1983-11-20,63.9,55.7,9.2,7.9,22.9,72.9,60.1,0.22
14240,722900,1983-11-21,60.4,46.5,10.9,14.1,29.9,69.1,54.0,0.48
14390,722900,1984-04-19,63.9,53.8,11.4,11.5,24.1,68.0,60.1,0.11
98918,725910,1985-04-21,52.5,41.2,13.0,11.6,18.1,60.1,46.9,0.03
71115,724800,1987-12-16,30.1,17.0,13.3,15.0,33.0,33.1,23.0,0.01
75304,724800,1992-12-30,33.1,24.6,42.8,6.3,19.8,41.9,23.0,0.35
73152,724800,1994-03-25,38.0,32.4,8.3,6.9,23.9,54.9,32.0,0.08
49910,723815,1999-10-07,68.3,37.8,29.4,12.2,35.9,82.2,55.2,0.00
49931,723815,1999-10-28,71.9,42.1,24.9,13.4,35.0,85.5,60.8,0.00
55442,723830,1999-10-04,74.6,39.0,10.0,13.4,23.9,82.0,55.9,0.00


In [244]:
df_subsetted = pd.read_csv('../data/preprocessed/CA/CA_1981-2015_subsetted2.csv', parse_dates=['YEARMODA'])

In [638]:
df_subsetted.head()

,Unnamed: 0,STN,YEARMODA,TEMP,PRCP,GUST,WDSP,MXSPD
0,0,691414,1985-04-24,54.1,0,15.0,5.6,9.9
1,1,691414,1985-04-25,42.9,NaN,25.1,12.6,19.0
2,2,691414,1985-04-26,43.0,NaN,20.0,8.8,15.9
3,3,691414,1985-04-27,42.4,NaN,53.0,16.3,29.9
4,4,691414,1985-04-28,40.5,NaN,40.0,14.0,27.0


In [246]:
df_subsetted.count()

Unnamed: 0    1370304
STN           1370304
YEARMODA      1370304
TEMP          1370304
PRCP          1341677
GUST           386697
WDSP          1342008
MXSPD         1338075
dtype: int64

# El nino index

In [212]:
oci = pd.read_csv('../data/raw/ONI.txt', sep=r'\t')

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/io/parsers.py:648: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.



In [213]:
oci['index'] = oci['Type'].map({'VSL':-2, 'SL':-1.5, 'ML':-1, 'WL':-0.5, 'N':0, 'WE':0.5, 'ME':1, 'SE':1.5, 'VSE':2})

<img src="../figures/elnino.png" />

#What El nino looks like in LA

<img src="http://www.trbimg.com/img-55d635df/turbine/la-me-g-0820-el-nino-rainfall-comparison-20150820/750/750x422" />

Source: http://www.latimes.com/local/lanow/la-me-ln-el-nino-temperatures-new-record-20151117-story.html

In [249]:
from plotly import plotly as ply
from plotly import graph_objs as go

In [641]:
tt = la.set_index('YEARMODA')[['PRCP']]['1997-7-1':'1998-6-30']
tt2 = la.set_index('YEARMODA')[['PRCP']]['2014-7-1':'2015-6-30']
rr = tt.resample('M') 
rr2 = tt2.resample('M')

In [643]:
traces = []
traces.append(go.Scatter(
    x=tt.index.map(lambda x:x.strftime('%b %d')),
    y=tt['PRCP'],
    name='1997-1998'))
traces.append(go.Scatter(
    x=tt2.index.map(lambda x:x.strftime('%b %d')),
    y=tt2['PRCP'],
    name='2014-2015'))
traces.append(go.Scatter(
    x=rr.index.map(lambda x:x.strftime('%b %d')),
    y=rr['PRCP'],
    name='1997-1998 monthly'))
traces.append(go.Scatter(
    x=rr2.index.map(lambda x:x.strftime('%b %d')),
    y=rr2['PRCP'],
    name='2014-2015 monthly'))
layout= go.Layout(title='Precipitation in 1997-1998 vs 2014-2015 in LA')
ply.iplot({'data':traces, 'layout':layout}, filename='WeatherData/test1')

There's a huge difference in precipitation between 1997-1998 and 2014-2015, but only obvious in Feb.

## Precipitation and El nino (Higher index means stronger el nino)

In [273]:
traces = []
traces.append(go.Scatter(x=haha['index'], y=haha['PRCP'], text=haha.index, mode='markers', 
                         marker = dict(color=haha['index'], colorscale='Viridis')))
ply.iplot(traces, filename='WeatherData/precipitation')

Positive Correlation between precipitation and el nino. (at least in LA)

## Temperature and El nino

In [289]:
traces = []
traces.append(go.Scatter(x=haha['PRCP'], y=haha['TEMP'], text=haha.index, mode='markers', 
                         marker = dict(color=haha['index'])))
ply.iplot(traces, filename='WeatherData/test7')

TEMP is not correlated with el nino

#PCA

In [484]:
all2 = all_shifted.copy()

In [485]:
all2['year'] = all2.index.year

In [486]:
all2.head()

,STN,PRCP,WDSP,MXSPD,TEMP,year
YEARMODA,,,,,,
1984-08-31,697534,0,8.4,9.9,71.2,1984
1984-08-31,697534,0,8.7,9.9,70.7,1984
1984-05-31,722810,0,2.8,9.9,67.2,1984
1984-05-31,722810,0,3.0,7.0,67.8,1984
1984-05-31,722810,0,1.9,8.0,68.1,1984


In [488]:
all2_years = all2.groupby(['STN', 'year']).mean()
all2_years = all2_years.reset_index()

In [500]:
all2_weeks = all2.groupby('STN').resample('W')
all2_weeks = all2_weeks.reset_index()

In [603]:
all2_max = all2_weeks.set_index('YEARMODA').groupby('STN')[['MXSPD', 'PRCP', 'WDSP']].resample('A', 'max').reset_index()

In [604]:
all2_years.head()

,STN,year,PRCP,WDSP,MXSPD,TEMP
0,690020,1989,0.031148,2.032787,4.824590,50.537705
1,690020,1990,0.013043,2.556522,5.895652,55.636232
2,690020,1991,0.061842,2.457895,5.342105,56.144737
3,690020,1992,0.209211,3.143590,6.937662,52.902564
4,690020,1993,0.089265,3.427941,6.886765,52.938235


In [605]:
all2_max.head()

,STN,YEARMODA,MXSPD,PRCP,WDSP
0,690020,1989-12-31,5.687500,0.057917,2.491667
1,690020,1990-12-31,6.642105,0.025714,2.884211
2,690020,1991-12-31,6.578947,0.225789,3.200000
3,690020,1992-12-31,8.984211,0.367895,4.336842
4,690020,1993-12-31,8.210526,0.186316,3.994737


In [606]:
all2_pivot = all2_max.pivot(index='YEARMODA', columns='STN')

In [607]:
all2_notnull = all2_pivot.dropna(axis=1)

In [625]:
all2_notnull.head(20)

MXSPD                                                         \
STN            722860     722868     722880     722886     722900     722904   
YEARMODA                                                                       
1981-12-31  12.277419  13.014286  10.357692  13.725806  12.819355   4.900000   
1982-12-31  11.190323  12.153333  10.835484  13.540000  13.370968   8.600000   
1983-12-31  11.717241  13.883333  11.239286  13.416667  12.189655  13.048276   
1984-12-31  12.738710  12.780000  11.425000  13.306897  12.635484  12.603226   
1985-12-31  10.909677  12.125000  11.628571  12.450000  12.639286  13.835484   
1986-12-31  13.139286  14.039286  10.846667  12.067857  13.737037  14.064286   
1987-12-31  10.658065  12.200000  12.277778  14.889655  13.045161  12.683871   
1988-12-31   9.883333  14.982143  11.722581  14.754839  12.200000  13.221429   
1989-12-31  12.332258  13.325000  13.160714  13.922581  12.303571  13.625000   
1990-12-31  12.074194  15.489655   9.900000  15.580645  12.122581  12.540000   
1991-12-31  12.419355  13.040000  13.058621  16.369565  13.750000  13.993103   
1992-12-31  11.376667  13.551852  12.546429  16.283333  13.538710  13.783333   
1993-12-31  13.728571  13.445161  11.560714  15.666667  12.778571  13.110714   
1994-12-31  13.720000  15.093333  12.706667  16.816667  14.229032  13.170968   
1995-12-31  11.517241  16.882759  13.251613  15.164516  12.031034  12.268966   
1996-12-31  12.030000  15.525000  16.053571  13.832143  11.950000  11.412903   
1997-12-31  13.987097  13.619355  16.577778  15.080645  15.089286  13.389286   
1998-12-31  10.632258  11.887097  11.383871  14.790323  11.525806  10.538710   
1999-12-31  12.596774  12.348276  11.117241  14.206452  13.200000  13.313793   
2000-12-31  11.485714  13.071429  11.750000  12.396774  12.689286  11.632143   

                                                          ...          WDSP  \
STN            722906     722907     722909     722925    ...        725940   
YEARMODA                                                  ...                 
1981-12-31  11.877419  11.380645  12.208333  12.893548    ...      8.338095   
1982-12-31  13.167742  11.500000  14.104348  13.839286    ...     11.500000   
1983-12-31  11.480000  12.455172   9.470833  13.703333    ...      5.957143   
1984-12-31  12.051613  10.294118  10.968000  14.093548    ...     10.900000   
1985-12-31  12.625000  10.790323  11.800000  11.242857    ...      9.276471   
1986-12-31  14.010714  11.628571  13.019231  10.416000    ...      8.430000   
1987-12-31  13.719355  10.293548  11.540000  10.625000    ...      9.040909   
1988-12-31  11.848387  11.359259  12.495652  12.051852    ...      7.458333   
1989-12-31  13.239286  11.635714  11.273913  14.800000    ...      6.429630   
1990-12-31  12.539286  11.339286  12.252000  17.166667    ...      6.021739   
1991-12-31  13.634483  11.962069  12.829167  17.000000    ...      8.740909   
1992-12-31  14.616129  11.692857  12.837500  11.160000    ...      6.883333   
1993-12-31  12.393548  11.410714  13.782609  11.780952    ...      6.854545   
1994-12-31  16.325806  11.220000  13.750000  14.566667    ...      7.604167   
1995-12-31  13.165517  10.671429  11.750000  12.604762    ...      7.125000   
1996-12-31  13.282143  11.435714  12.852174  16.563158    ...      7.547368   
1997-12-31  16.867857  12.085714  14.978261  19.985714    ...     10.089474   
1998-12-31  14.123333  10.400000  11.380000  16.186364    ...      8.388000   
1999-12-31  13.500000  11.551724  11.728571  14.181818    ...      7.071429   
2000-12-31  13.635714  11.150000  12.435000  13.855000    ...      6.416667   

                                                                          \
STN           725945     725946    725957     745160    746120    747020   
YEARMODA                                                                   
1981-12-31  6.445161   9.764516  4.754839   5.280645  5.300000  4.170968   
1982-12-31  8.235714  11.271429  4.670968   6.867857 

In [608]:
len(all2_notnull.columns)

161

In [609]:
from sklearn.decomposition import PCA

In [626]:
pca = PCA(n_components=3)

In [627]:
projected = pca.fit_transform(all2_notnull)

In [628]:
all2_max.head()

,STN,YEARMODA,MXSPD,PRCP,WDSP
0,690020,1989-12-31,5.687500,0.057917,2.491667
1,690020,1990-12-31,6.642105,0.025714,2.884211
2,690020,1991-12-31,6.578947,0.225789,3.200000
3,690020,1992-12-31,8.984211,0.367895,4.336842
4,690020,1993-12-31,8.210526,0.186316,3.994737


In [629]:
oci_numbers = oci.set_index('From')['index'][all2_notnull.index.year]

In [630]:
oci_adjusted = oci_numbers.replace({-2:0, -1.5:0, -1:0, -0.5:0, 0.5:0})

In [631]:
trace0 = go.Scatter3d(
    x=projected[:,0],
    y=projected[:,1],
    z=projected[:,2],
    name='Projected',
    mode='markers+text',
    text=all2_notnull.index.year,
    marker=go.Marker(color=oci_adjusted, 
                     size=15,
                    )
)
layout = go.Layout(
    title="California transformed PCA"
)
figure = go.Figure(data=[trace0], layout=layout)
ply.iplot(figure, filename='WeatherData/sandiego_PCA2')

In [621]:
ply.image.save_as(figure, '../figures/California PCA projected max.png')

In [645]:
pca.explained_variance_ratio_

array([ 0.25400467,  0.20993609,  0.09388   ])

#K means

In [653]:
df_monthly = df_subsetted.groupby('STN').apply(lambda x: x.set_index('YEARMODA')[['PRCP', 'MXSPD', 'WDSP']].resample('M'))
df_monthly = df_monthly.reset_index()

In [656]:
def extract_novembers(df):
    df_timed = df.set_index('YEARMODA')[['PRCP', 'MXSPD', 'WDSP']]
    return df_timed[df_timed.index.month == 11]

In [657]:
df_novembers = df_monthly.groupby('STN').apply(extract_novembers)
df_novembers = df_novembers.reset_index()

In [659]:
df_novembers.head(10)

,STN,YEARMODA,PRCP,MXSPD,WDSP
0,690020,1990-11-30,0.000000,6.382353,2.882353
1,690020,1991-11-30,0.000000,5.022222,2.561111
2,690020,1992-11-30,0.000000,5.950000,2.611111
3,690020,1993-11-30,0.018667,7.026667,3.526667
4,690020,1994-11-30,0.045455,8.395455,4.072727
5,690020,1995-11-30,0.015385,6.284615,3.153846
6,690020,1996-11-30,NaN,NaN,NaN
7,690070,1991-11-30,NaN,NaN,NaN
8,690070,1992-11-30,0.002105,10.326316,4.126316
9,690140,1989-11-30,0.000000,10.058065,4.535484


In [660]:
dff = df_novembers.set_index('YEARMODA')['1990':'2015']
dff = dff.reset_index()

In [663]:
all_data = dff.pivot_table(index=['YEARMODA'], columns=['STN']).dropna(axis=1)
all_data.head()

PRCP                                                      \
STN           690150 722810    722860 722868 722880 722885 722886 722897   
YEARMODA                                                                   
1990-11-30  0.000000  0.000  0.010667      0      0      0      0      0   
1991-11-30  0.000000  0.000  0.000667      0      0      0      0      0   
1992-11-30  0.000000  0.000  0.000000      0      0      0      0      0   
1993-11-30  0.006897  0.014  0.015484      0      0      0      0      0   
1994-11-30  0.000000  0.000  0.010333      0      0      0      0      0   

                                  ...         WDSP                      \
STN           722900    722903    ...       725945    725946    725955   
YEARMODA                          ...                                    
1990-11-30  0.028000  0.006552    ...     4.506667  6.470000  3.393333   
1991-11-30  0.001667  0.000690    ...     3.833333  7.343333  3.792857   
1992-11-30  0.001333  0.404333    ...     5.383333  6.796667  2.653333   
1993-11-30  0.029677  0.318000    ...     5.912903  7.029032  3.309677   
1994-11-30  0.015333  0.349000    ...     6.570000  8.476667  4.826667   

                                                                        \
STN           725957    745160    746120    747020    747185    747187   
YEARMODA                                                                 
1990-11-30  4.226667  6.213333  4.163636  4.980000  6.233333  5.446667   
1991-11-30  4.216667  6.066667  3.083333  5.123333  5.990000  4.330000   
1992-11-30  3.466667  5.710000  2.117391  3.700000  5.096667  4.113333   
1993-11-30  4.312903  8.874194  3.368000  3.764516  5.261290  4.609677   
1994-11-30  3.423333  9.173333  4.218182  3.460000  7.056667  4.010000   

                      
STN           747188  
YEARMODA              
1990-11-30  8.273333  
1991-11-30  6.766667  
1992-11-30  6.236667  
1993-11-30  5.532258  
1994-11-30  6.993333  

[5 rows x 197 columns]

In [664]:
import numpy as np

mse = []
# all_data = pd.read_csv('CA_notnull.csv', header=[0,1], index_col=0)
# all_data.index = pd.to_datetime(all_data.index)

#el nino years
df_15 = all_data[all_data.index.year == 2015].values
df_97 = all_data[all_data.index.year == 1997].values

df_02 = all_data[all_data.index.year == 2002].values
df_09 = all_data[all_data.index.year == 2009].values

#non el nino
df_90 = all_data[all_data.index.year == 1990].values


val1 = np.sum((df_15 - df_97)**2)
mse.append(val1)
val2 = np.sum((df_15 - df_02)**2)
mse.append(val2)
val3 = np.sum((df_15 - df_09)**2)
mse.append(val3)
val4 = np.sum((df_15 - df_90)**2)
mse.append(val4)

In [666]:
pca3 = PCA(n_components=2)
projected = pca3.fit_transform(all_data)
projected

array([[ 10.0646219 ,  -9.55781212],
       [ 11.55095049,  -8.90832978],
       [  8.06138439, -11.61618962],
       [  9.5948781 ,  -4.65299462],
       [ 21.63525871,   2.84868553],
       [ -7.18489677,  -6.87992969],
       [  9.28240383,   0.49930925],
       [  1.38546784,  -2.83563511],
       [  7.43868647,  10.15509315],
       [ -4.61814651,  -0.25988937],
       [ -2.2262206 ,  -3.0296178 ],
       [  6.52604248,   7.96991539],
       [  1.60636546,  -2.11032343],
       [ -4.97543522,   0.77982686],
       [ -3.48480121,  -3.63588304],
       [ -3.96425086,  -0.08440152],
       [ -0.29513694,   8.28322089],
       [-12.98383178,  -4.13400644],
       [ -8.25597417,  -4.72194649],
       [ -8.66374642,   0.32712538],
       [  1.22445276,   2.05653421],
       [ -4.36702104,   4.30818471],
       [-11.38785647,   2.91151587],
       [-10.65337112,  -3.81345817],
       [ -5.41140166,  -0.57580133],
       [  0.10157834,  26.67680726]])

In [667]:
# two dimensional projection
trace0 = go.Scatter(
    x=projected[:,0],
    y=projected[:,1],
    name='Projected',
    mode='markers+text',
    text=all_data.index.year,
)
layout = go.Layout(
    title="California transformed PCA"
)
figure = go.Figure(data=[trace0], layout=layout)
ply.iplot(figure, filename='WeatherData/sandiego_PCA22')

In [681]:
import scipy.cluster.vq as CL
import numpy as np

#number of centroids
numK = 5
centroids, sq = CL.kmeans(np.array(projected), numK)

#compute distance of each sample to the centroids
dist = [(np.sum((np.array(projected) - np.array([centroids[i]]))**2,1)) for i in range(numK)]

#assign sample to closest cluster
class_assigned = [(np.where(np.transpose(dist)[i]==min(np.transpose(dist)[i])))[0][0] for i in range(projected.shape[0])]
clustered = [[projected[i] for i, clu in enumerate(class_assigned) if clu == k*1.] for k in range(numK)]

# plotting each class
traces = []
for k in range(numK):
    trace = go.Scatter(x=np.transpose(clustered[k])[0], y=np.transpose(clustered[k])[1],
                      mode='markers')
    traces.append(trace)

In [682]:
layout = go.Layout(showlegend=False)
ply.iplot({'data':traces, 'layout':layout})

In [683]:
pca22 = PCA(n_components=3)
projected2 = pca22.fit_transform(all_data)

# 3 dimensional projection
trace1= go.Scatter3d(x=projected2[:,0],
                    y=projected2[:,1],
                    z=projected2[:,2],
                    mode='markers+text',
                    text=all_data.index.year
                    )
data = [trace1]
ply.iplot(data,filename='simple-3d-scatter')

#California Visualization

## Normal

<img src="../figures/CA_Normal.png" />

## El Nino

<img src="../figures/CA_EL_NINO.png" />

## La Nina

<img src="../figures/CA_La_Nina.png" />